# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProjectExperiment'
project_folder = './capstone_automl_step'
experiment=Experiment(ws, experiment_name)

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded...................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
In this project we use the very popular heart-disease dataset that can be found on kaggle or the UCI ML repo. 

I use a subset of data with 14 features and as the label the "DEATH_EVENT" that indicates the death of the patient in a given period of time.

In [5]:
found = False
key = "Cleveland-Heart-Disease-Dataset"
description_text = "UCI ML Repo Dataset on non-invasive diagnosis of heart disease"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        example_data = 'https://raw.githubusercontent.com/sierraechobravoindia/udacity-capstone-ml-eng-nd/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset, 
                             label_column_name="DEATH_EVENT",  
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProjectExperiment,AutoML_f28929f9-9212-4c32-92bb-f5d5255b7b4f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion()

{'runId': 'AutoML_f28929f9-9212-4c32-92bb-f5d5255b7b4f',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-05-16T12:24:28.397989Z',
 'endTimeUtc': '2023-05-16T12:47:04.415365Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneProjectExperiment","subscription_id":"eb2fe91a-2797-4c53-bd12-b644f041e7ef","resource_group":"rg-art-ta-cfp-docker","workspace_name":"ml_playground","region":"westeurope","compute_target":"compute-clu

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

for metrics in best_run_metrics:
    print(metrics, best_run_metrics[metrics])

print(fitted_model)

accuracy 0.882994350282486
balanced_accuracy 0.8496130952380951
AUC_weighted 0.9069003552971576
precision_score_micro 0.882994350282486
f1_score_weighted 0.8785205518614699
average_precision_score_macro 0.8931823655735709
AUC_micro 0.9116662357560088
log_loss 0.46509202947114286
recall_score_micro 0.882994350282486
AUC_macro 0.9069003552971576
recall_score_macro 0.8496130952380951
average_precision_score_weighted 0.9177225884366529
average_precision_score_micro 0.9134732011234743
precision_score_weighted 0.8946960450672214
precision_score_macro 0.890782925201823
f1_score_macro 0.8578354176828625
matthews_correlation 0.7374155346601782
recall_score_weighted 0.882994350282486
norm_macro_recall 0.6992261904761905
f1_score_micro 0.882994350282486
weighted_accuracy 0.9052696285543416
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_f28929f9-9212-4c32-92bb-f5d5255b7b4f_40/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_f28929f9-9212-4c32-92bb-f5d5255b7b4

In [22]:
#TODO: Save the best model

best_run.register_model(model_name = 'capstone-automl', model_path='./outputs')

Model(workspace=Workspace.create(name='ml_playground', subscription_id='eb2fe91a-2797-4c53-bd12-b644f041e7ef', resource_group='rg-art-ta-cfp-docker'), name=test, id=test:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
# Download score.py and env file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH,'./automl_env.yml')

In [11]:
#Register the model
model_name = "capstone_automl"
description = 'Best AutoML Model for the udacity ML ND capstone project'
model = remote_run.register_model(model_name = model_name, description = description)

In [13]:
# deploy model for inference 
script_file_name= 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'heart-failure-inference'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-16 13:11:18+00:00 Creating Container Registry if not exists.
2023-05-16 13:11:19+00:00 Use the existing image.
2023-05-16 13:11:20+00:00 Submitting deployment to compute.
2023-05-16 13:11:26+00:00 Checking the status of deployment heart-failure-inferene..
2023-05-16 13:13:53+00:00 Checking the status of inference endpoint heart-failure-inferene.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [16]:
# Enable Application Insights
aci_service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
import requests
import json

scoring_uri = aci_service.scoring_uri

data = {"data":
        [
          {
            "age": 98.0,
            "anaemia": 0,
            "creatinine_phosphokinase": 3990,
            "diabetes": 1,
            "ejection_fraction": 29,
            "high_blood_pressure": 1,
            "platelets": 342721.29,
            "serum_creatinine": 1.0,
            "serum_sodium": 109,
            "sex": 0,
            "smoking": 1,
            "time": 11
          },
          {
            "age": 79.0,
            "anaemia": 0,
            "creatinine_phosphokinase": 4094,
            "diabetes": 0,
            "ejection_fraction": 21,
            "high_blood_pressure": 0,
            "platelets": 414792.12,
            "serum_creatinine": 1.3,
            "serum_sodium": 121,
            "sex": 1,
            "smoking": 0,
            "time": 9
          },
      ]
    }


input_data = json.dumps(data)
with open("data.json", "w") as f:
    f.write(input_data)

headers = {'Content-Type': 'application/json'}

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1, 1]}


In [22]:
print(f"State {aci_service.state}")
print(f"Swagger URL {aci_service.swagger_uri}")
print(f"Inference URL {aci_service.scoring_uri}")

State Healthy
Swagger URL http://b2ebc8c4-abac-4d01-bb69-332337b87bac.westeurope.azurecontainer.io/swagger.json
Inference URL http://b2ebc8c4-abac-4d01-bb69-332337b87bac.westeurope.azurecontainer.io/score


TODO: In the cell below, print the logs of the web service and delete the service

In [23]:
#Print the logs of the deployed service
aci_service.get_logs()

'2023-05-16T13:16:48,720046465+00:00 - rsyslog/run \n2023-05-16T13:16:48,719679364+00:00 - iot-server/run \n2023-05-16T13:16:48,727058076+00:00 - gunicorn/run \n2023-05-16T13:16:48,737266793+00:00 | gunicorn/run | \n2023-05-16T13:16:48,740942899+00:00 | gunicorn/run | ###############################################\n2023-05-16T13:16:48,751192916+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-05-16T13:16:48,754590021+00:00 | gunicorn/run | ###############################################\n2023-05-16T13:16:48,760289930+00:00 | gunicorn/run | \n2023-05-16T13:16:48,769788046+00:00 | gunicorn/run | \n2023-05-16T13:16:48,774719754+00:00 - nginx/run \n2023-05-16T13:16:48,791182281+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230120.v2\n2023-05-16T13:16:48,793817785+00:00 | gunicorn/run | \n2023-05-16T13:16:48,797710992+00:00 | gunicorn/run | \n2023-05-16T13:16:48,802692100+00:00 | gunicorn/run | PATH environment variable

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
